In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy import sparse
from sklearn.cross_validation import train_test_split

/Users/michellehsu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
y = pd.read_csv('triptype.csv', header= None)
X = sparse.load_npz('df_sparse_matrix.npz')

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Logistic Regression

### Logistic Regression Without tuning

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)

In [6]:
error_lr = np.mean(pred_lr[:, np.newaxis] != y_test)
print(error_lr)

0    0.309412
dtype: float64


### Logistic Regression With tuning

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [8]:
parameters = {'C':[1e-2, 1e-1, 1, 10, 100]}
lr_cv = GridSearchCV(LogisticRegression(penalty='l1'), cv = StratifiedKFold(n_splits=5, shuffle = True), param_grid = parameters, n_jobs = -1)
lr_cv.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.01, 0.1, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [9]:
pred_lr_cv = lr_cv.predict(X_test)
error_lr_cv = np.mean(pred_lr_cv[:, np.newaxis] != y_test)
print(error_lr_cv)

0    0.306847
dtype: float64


## SVM

### SVM Without tuning

In [10]:
from sklearn.svm import LinearSVC

In [11]:
svc = LinearSVC()
svc.fit(X_train, y_train)
pred_svc = svc.predict(X_test)

In [12]:
error_svc = np.mean(pred_svc[:, np.newaxis] != y_test)
print(error_svc)

0    0.329722
dtype: float64


### SVM With tuning

In [13]:
svc_cv = GridSearchCV(LinearSVC(penalty='l2'), cv = StratifiedKFold(n_splits=5, shuffle = True), param_grid = parameters, n_jobs = -1)
svc_cv.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.01, 0.1, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [14]:
pred_svc_cv = svc_cv.predict(X_test)
error_svc_cv = np.mean(pred_svc_cv[:, np.newaxis] != y_test)
print(error_svc_cv)

0    0.309621
dtype: float64
